## Imports

In [10]:
import numpy as np
import pandas as pd

## Purpose

We want to explore 3 well-known stochastic processes:
- geometric brownian motion
- jump diffusion
- square-root diffusion

## Functions

If you want to end up doing Monte Carlo simulations, you will need a way to generate random numbers with different distributions.
Note that we're generating distributions with variance reduction techniques like antithetic paths and moment matching.

In [2]:

def sn_random_numbers(shape, antithetic=True, moment_matching=True, fixed_seed=False):
        '''
        Returns array of shape shape with (pseudo)random numbers that are standard normally distributed.
        
        Parameters
        ==========
        shape : tuple (o, n, m)
            generation of array with shape (o, n, m)
        antithetic : Boolean
            generation of antithetic variates
        moment_matching : Boolean
            matching of first and second moments
        fixed_seed : Boolean
            flag to fix the seed
            
        Results
        =======
        ran : (o, n, m) array of (pseudo)random numbers
        '''
        if fixed_seed:
            np.random.seed(1000)
        if antithetic:
            ran = np.random.standard_normal((shape[0], shape[1], shape[2] / 2))
            ran = np.concatenate((ran, -ran), axis=2)
        else:
            ran = np.random.standard_normal(shape)
        if moment_matching:
            ran = ran - np.mean(ran)
            ran = ran / np.std(ran)
        if shape[0] == 1:
            return ran[0]
        else:
            return ran

In [3]:
snrn = sn_random_numbers((2, 2, 2,), antithetic=False, moment_matching=False, fixed_seed=True)

In [4]:
snrn

array([[[-0.8044583 ,  0.32093155],
        [-0.02548288,  0.64432383]],

       [[-0.30079667,  0.38947455],
        [-0.1074373 , -0.47998308]]])

In [6]:
snrn_mm = sn_random_numbers((2,3,2), antithetic=False, moment_matching=True, fixed_seed=True)

In [7]:
snrn_mm

array([[[-1.47414161,  0.67072537],
        [ 0.01049828,  1.28707482],
        [-0.51421897,  0.80136066]],

       [[-0.14569767, -0.85572818],
        [ 1.19313679, -0.82653845],
        [ 1.3308292 , -1.47730025]]])

In [8]:
snrn_mm.mean()

3.700743415417188e-17

In [9]:
snrn_mm.std()

1.0

## Generic Simulation Class

In [ ]:
class simulation_class(object):
    '''
    Base methods for simulation classes
    
    Attributes
    ==========
    name : string
        name of object
    mar_env : instance of market_environment
        market environment data for simulation
    corr : Boolean
        True if correlated with other model object
        
    Methods
    =======
    generate_time_grid :
        returns time grid for simulation
    get_instrument_values : 
        returns current instrument values (array)
    '''
    
    def __init__(self, name, mar_env, corr):
        try:
            self.name = name
            self.pricing_date = mar_env.pricing_date
            self.initial_value = mar_env.get_constant('initial_value')
            self.volatility = mar_env.get_constant('volatility')
            self.final_date = mar_env.get_constant('final_date')
            self.currency = mar_env.get_constant('currency')
            self.frequency = mar_env.get_constant('frequency')
            self.paths = mar_env.get_constant('paths')
            self.discount_curve = mar_env.get_curve('discout_curve')
            try:
                # if time_grid in mar_env take this
                # (for portfolio valuation)
                self.time_grid = mar_env.get_list('time_grid')
            except:
                self.time_grid = None
            try:
                # if special dates exist, add them
                self.special_dates = mar_env.get_list('special_dates')
            except:
                self.special_dates = []
            self.instrument_values = None
            self.correlated = corr
            if corr is True:
                # only necessary in portfolio context when
                # risk factors are correlated
                self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
                self.rn_set = mar_env.get_list('rn_set')[self.name]
                self.random_numbers = mar_env.get_list('random_numbers')
                except:
                    print("Error parsing market environment")
    
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final-date
        # pandas date_range function
        # freq = 'B' for business day
        # 'W' for weekly, 'M' for monthly
        time_grid = pd.date_range(start=start, end=end
                                  , freq=self.frequency).to_pydatetime()
        time_grid = list(time_grid)
        # enhance time_grid by start, end, and special_dates
        if start not in time_grid:
            time_grid.insert(0, start)
            # insert start date if not in list
        if end not in time_grid:
            time_grid.append(end)
            # insert end date if not in list
        if len(self.special_dates) > 0:
            # add all special dates
            time_grid.extend(self.special_dates)
            # delete duplicates
            time_grid = list(set(time_grid))
            # sort list
            time_grid.sort()
        self.time_grid = np.array(time_grid)
        
    def get_instrument_values(self, fixed_seed=True):
        if self.instrument values is None:
            # only initiate simulation if no instrument values exist
            self.generate_paths(fixed_seed=fixed_seed, day_count=365)
        elif fixed_seed is False:
            # also initiate resimulation when fixed_seed is False
            self.generate_paths(fixed-seed=f
        